# Aula 13

Junção de tudo que foi feito ao longo do semestre

In [3]:
import pandas as pd
import requests
from io import StringIO
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


# Criação do dataFrame dos alunos
# ID do arquivo no Google Drive
file_id = '1S5Nl793vcL5ZPTGjzKaIEbwbLaDplvIP'

# URL modificada para forçar o download do arquivo
url = f"https://drive.google.com/uc?id={file_id}"

# Tentando obter o arquivo com requests
try:
    response = requests.get(url)
    response.raise_for_status()  # Lança um erro para respostas não-sucedidas
    # Usando StringIO para converter o texto em um arquivo em memória e, então, lendo com o Pandas
    csv_raw = StringIO(response.text)
    data = pd.read_csv(csv_raw)
except requests.RequestException as e:
    print(f"Erro ao acessar o arquivo: {e}")


#Transformando o PassengerId no índice
data = data.set_index("PassengerId")

#Criando uma função que encontra os pronomes de tratamentos dos passageiros
def extract_title(name):
    title_search = re.search(' ([A-Za-z]+)\.', name)
    if title_search:
        return title_search.group(1)
    return ""

#Aplicando a função na coluna de nomes, isto é, criando uma coluna só com os pronomes de tratamentos
data['Title'] = data['Name'].apply(extract_title)

#Substituindo os valores ausentes das idades pela mediana agrupada por sexo e classe de passageiro
data['Age'] = data.groupby(['Sex', 'Pclass'])['Age'].transform(lambda x: x.fillna(x.median()))

#Substituindo os valores ausentes do porto de embarque pelo valor que mais apareceu
data['Embarked'] = data['Embarked'].fillna('S')

#Substituindo os valores ausentes da cabine com base na Classe e no mapa do navio
for num in [1, 2, 3]:
    if num == 1:
        data.loc[data['Pclass'] == 1, 'Cabin'] = data.loc[data['Pclass'] == 1, 'Cabin'].fillna('ABC')
    elif num == 2:
        data.loc[data['Pclass'] == 2, 'Cabin'] = data.loc[data['Pclass'] == 2, 'Cabin'].fillna('DE')
    elif num == 3:
        data.loc[data['Pclass'] == 3, 'Cabin'] = data.loc[data['Pclass'] == 3, 'Cabin'].fillna('FG')
        
# Inicializando o LabelEncoder
label_encoder = LabelEncoder()

df_encoded = data.copy()

# Colunas a serem transformadas
columns_to_encode = ['Sex', 'Cabin', 'Title', 'Embarked']

# Aplicando LabelEncoder às colunas categóricas
for column in columns_to_encode:
    df_encoded[column] = label_encoder.fit_transform(df_encoded[column])
    
#Retirando as colunas Nome e Ticket
df_encoded = df_encoded.drop(['Name','Ticket'], axis=1)

x_train, x_test, y_train, y_test = train_test_split(df_encoded.drop(['Survived'],axis=1),
                                                     df_encoded['Survived'],
                                                     test_size = 0.3,
                                                     random_state = 1234)

[{'treino':x_train.shape},{'teste':x_test.shape}]

rf = RandomForestClassifier(n_estimators=1000,
                            criterion='gini',
                            max_depth = 5)

rf.fit(x_train,y_train)

prob = rf.predict_proba(df_encoded.drop('Survived',axis =1))[:,1]
cla = rf.predict(df_encoded.drop('Survived',axis = 1))

data['Probability'] = prob
data['Classification'] = cla

def prever_sobrevivencia(model, attributes, df_template):
    """
    Função para prever a probabilidade de sobrevivência com base em atributos fornecidos.
    
    Parâmetros:
    - model: O modelo treinado (Random Forest).
    - attributes: Um dicionário contendo os atributos necessários para a predição.
    - df_template: DataFrame usado como template para manter a ordem e colunas corretas.
    
    Retorna:
    - probabilidade: A probabilidade de sobrevivência.
    - classificacao: 1 se a pessoa provavelmente sobreviveu, 0 caso contrário.
    """
    # Adicionar placeholders para 'Cabin' e 'Title' nos atributos fornecidos
    attributes['Cabin'] = 0
    attributes['Title'] = 0
    
    # Crie um DataFrame com os atributos fornecidos
    df = pd.DataFrame([attributes])
    
    # Colunas necessárias para a predição, baseadas no df_template
    columns_to_use = df_template.drop(['Survived', 'Probability', 'Classification', 'Name', 'Ticket'], axis=1, errors='ignore').columns
    
    # Reindexe para garantir que todas as colunas estejam presentes
    df = df.reindex(columns=columns_to_use, fill_value=0)
    
    # Faça a previsão
    probabilidade = model.predict_proba(df)[:, 1][0]
    classificacao = model.predict(df)[0]
    
    return probabilidade, classificacao

In [4]:
# Exemplo de uso da função
atributos_exemplo = {
    'Pclass': 2,
    'Sex': 0, 
    'Age': 40,
    'SibSp': 1,
    'Parch': 0,
    'Fare': 50.25,
    'Embarked': 2 
}

probabilidade, classificacao = prever_sobrevivencia(rf, atributos_exemplo, df_encoded)
print(f'Probabilidade de Sobrevivência: {probabilidade:.2f}')
print(f'Classificação: {"Sobreviveu" if classificacao == 1 else "Não Sobreviveu"}')


Probabilidade de Sobrevivência: 0.84
Classificação: Sobreviveu
